# Hoán đổi token bằng cách sử dụng API REST của Marlowe Runtime

***Trước khi chạy sổ ghi chép này, bạn có thể muốn sử dụng chức năng "xóa đầu ra" của Jupyter để xóa kết quả thực hiện trước đó của sổ ghi chép này. Điều đó sẽ làm rõ hơn những gì đã được thực thi trong phiên hiện tại.***

Ví dụ về hợp đồng token-swap là một hợp đồng Marlowe đơn giản cho phép các bên giao dịch ADA lấy token.

[Một video hoạt động thông qua sổ tay Jupyter này.](https://youtu.be/sSrVCRNoytU)

Bạn có thể đặt câu hỏi về Marlowe trong [kênh #ask-marlowe trên IOG Discord](https://discord.com/channels/826816523368005654/936295815926927390) hoặc đăng các vấn đề với bài học này lên [danh sách các vấn đề của Bộ công cụ dành cho người mới bắt đầu Marlowe kho lưu trữ github.](https://github.com/input-output-hk/marlowe-starter-kit/issues).

Trong phần trình diễn này, chúng ta sử dụng API REST của Marlowe Runtime\, được cung cấp qua `marlowe-web-server`, để chạy hợp đồng này trên mạng thử nghiệm công khai `preprod` của Cardano\. Hợp đồng Marlowe có thể sử dụng địa chỉ hoặc vai trò token để ủy quyền: ở đây chúng ta sử dụng vai trò token và chúng ta có Marlowe Runtime đúc chúng.

Trong [Marlowe Playground](https://play.marlowe-finance.io/), hợp đồng có dạng như sau ở định dạng Blockly.

![Marlowe contract for swapping tokens](images/swap-playground.png)

Ở định dạng Marlowe, nó xuất hiện dưới dạng
```
When
    [Case
        (Deposit
            (Role "Ada provider")
            (Role "Ada provider")
            (Token "" "")
            (MulValue
                (Constant 1000000)
                (ConstantParam "Amount of Ada")
            )
        )
        (When
            [Case
                (Deposit
                    (Role "Dollar provider")
                    (Role "Dollar provider")
                    (Token "9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe" "Djed_testMicroUSD")
                    (ConstantParam "Amount of dollars")
                )
                (Pay
                    (Role "Ada provider")
                    (Party (Role "Dollar provider"))
                    (Token "" "")
                    (MulValue
                        (Constant 1000000)
                        (ConstantParam "Amount of Ada")
                    )
                    (Pay
                        (Role "Dollar provider")
                        (Party (Role "Ada provider"))
                        (Token "9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe" "Djed_testMicroUSD")
                        (ConstantParam "Amount of dollars")
                        Close 
                    )
                )]
            (TimeParam "Timeout for dollar deposit")
            Close 
        )]
    (TimeParam "Timeout for Ada deposit")
    Close
```

## Chuẩn bị

Xem [Lesson 0. Preliminaries](00-preliminaries.md) để biết thông tin về cách thiết lập môi trường của một người để sử dụng hướng dẫn này.

Bài học giả định rằng các biến môi trường sau đây đã được thiết lập.
- `CARDANO_NODE_SOCKET_PATH`: vị trí socket của node Cardano.
- `CARDANO_TESTNET_MAGIC`: số magic testnet.
- `MARLOWE_RT_WEBSERVER_HOST`: Địa chỉ IP của máy chủ web Marlowe Runtime.
- `MARLOWE_RT_WEBSERVER_PORT`: Số cổng cho máy chủ web Marlowe Runtime.

Nó cũng giả định rằng các bên có địa chỉ, khóa ký và tiền.
- Người bán
    - [keys/lender.address](keys/lender.address): Địa chỉ Cardano của người bán
    - [keys/lender.skey](keys/lender.skey): vị trí ký file key cho người bán
- Người mua
    - [keys/borrower.address](keys/borrower.address): Địa chỉ Cardano cho người mua
    - [keys/borrower.skey](keys/borrower.skey): vị trí ký file key cho người mua
- Hòa giải viên
    - [keys/mediator.address](keys/mediator.address): Địa chỉ Cardano cho người hòa giải
    - [keys/mediator.skey](keys/mediator.skey): vị trí ký tệp khóa cho người hòa giải

### Truy cập vào node Cardano và Runtimes Marlowe

Nếu chúng ta đang sử dụng tiện ích mở rộng [demeter.run](https://demeter.run/), thì chúng ta đã có quyền truy cập vào Cardano Node và Marlowe Runtime. Các lệnh sau sẽ đặt các biến môi trường cần thiết để sử dụng triển khai docker cục bộ trên các cổng mặc định. Nó cũng sẽ thiết lập một số biến môi trường bổ sung.

In [1]:
if [[ -z "$MARLOWE_RT_WEBSERVER_PORT" ]]
then

  # Only required for `marlowe-cli` and `cardano-cli`.
  export CARDANO_NODE_SOCKET_PATH="$(docker volume inspect marlowe-starter-kit_shared | jq -r '.[0].Mountpoint')/node.socket"
  export CARDANO_TESTNET_MAGIC=1 # Note that preprod=1 and preview=2. Do not set this variable if using mainnet.

  # Only required for Marlowe Runtime REST API.
  export MARLOWE_RT_WEBSERVER_HOST="127.0.0.1"
  export MARLOWE_RT_WEBSERVER_PORT=3780

fi

# FIXME: This should have been inherited from the parent environment.
if [[ -z "$CARDANO_NODE_SOCKET_PATH" ]]
then
  export CARDANO_NODE_SOCKET_PATH=/ipc/node.socket
fi

# FIXME: This should have been set in the parent environment.
if [[ -z "$CARDANO_TESTNET_MAGIC" ]]
then
  export CARDANO_TESTNET_MAGIC=$CARDANO_NODE_MAGIC
fi

case "$CARDANO_TESTNET_MAGIC" in
  1)
    export "EXPLORER_URL=https://preprod.cardanoscan.io"
    ;;
  2)
    export "EXPLORER_URL=https://preview.cardanoscan.io"
    ;;
  *)
    # Use `mainnet` as the default.
    export "EXPLORER_URL=https://cardanoscan.io"
    ;;
esac

MARLOWE_RT_WEBSERVER_URL="http://$MARLOWE_RT_WEBSERVER_HOST":"$MARLOWE_RT_WEBSERVER_PORT"

echo "CARDANO_NODE_SOCKET_PATH = $CARDANO_NODE_SOCKET_PATH"
echo "CARDANO_TESTNET_MAGIC = $CARDANO_TESTNET_MAGIC"
echo "MARLOWE_RT_WEBSERVER_HOST = $MARLOWE_RT_WEBSERVER_HOST"
echo "MARLOWE_RT_WEBSERVER_PORT = $MARLOWE_RT_WEBSERVER_PORT"
echo "MARLOWE_RT_WEBSERVER_URL = $MARLOWE_RT_WEBSERVER_URL"

CARDANO_NODE_SOCKET_PATH = ~/.local/share/containers/storage/volumes/marlowe-starter-kit_shared/_data/node.socket
CARDANO_TESTNET_MAGIC = 1
MARLOWE_RT_WEBSERVER_HOST = 127.0.0.1
MARLOWE_RT_WEBSERVER_PORT = 3780
MARLOWE_RT_WEBSERVER_URL = http://127.0.0.1:3780


Lưu ý số magic mạng thử nghiệm:
- `preprod` = 1
- `preview` = 2

### Địa chỉ và tiền của Nhà cung cấp Ada

Kiểm tra xem địa chỉ và khóa đã được tạo cho nhà cung cấp ADA chưa. Nếu không, hãy xem phần "Tạo địa chỉ và khóa ký" trong Sơ bộ.

In [2]:
ADA_PROVIDER_SKEY=keys/lender.skey
ADA_PROVIDER_ADDR=$(cat keys/lender.address)
echo "ADA_PROVIDER_ADDR = $ADA_PROVIDER_ADDR"

ADA_PROVIDER_ADDR = addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


Kiểm tra xem nhà cung cấp ADA có ít nhất ba trăm ADA không.

In [3]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADA_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
81fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb51     1        1000000000 lovelace + TxOutDatumNone


Người ta có thể xem địa chỉ trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [4]:
echo "$EXPLORER_URL"/address/"$ADA_PROVIDER_ADDR"

https://preprod.cardanoscan.io/address/addr_test1vqd3yrtjyx49uld43lvwqaf7z4k03su8gf2x4yr7syzvckgfzm4ck


### Địa chỉ và tiền của Nhà cung cấp Dollar

Kiểm tra xem địa chỉ và khóa đã được tạo cho nhà cung cấp đô la chưa. Nếu không, hãy xem phần "Tạo địa chỉ và khóa ký" trong Sơ bộ.

In [5]:
USD_PROVIDER_SKEY=keys/borrower.skey
USD_PROVIDER_ADDR=$(cat keys/borrower.address)
echo "USD_PROVIDER_ADDR = $USD_PROVIDER_ADDR"

USD_PROVIDER_ADDR = addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d


Kiểm tra để đảm bảo rằng nhà cung cấp đô la có ít nhất một trăm USD token.

In [6]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$USD_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
81fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb51     2        1000000000 lovelace + TxOutDatumNone
81fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb51     3        2000000 lovelace + 100000000 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe.446a65645f746573744d6963726f555344 + TxOutDatumNone


Người ta có thể xem địa chỉ trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [7]:
echo "$EXPLORER_URL"/address/"$USD_PROVIDER_ADDR"

https://preprod.cardanoscan.io/address/addr_test1vpy4n4peh4suv0y55yptur0066j5kds8r4ncnuzm0vpzfgg0dhz6d


## Thiết kế hợp đồng

Bạn có thể tải xuống hợp đồng hoán đổi từ [Marlowe Playground](https://play.marlowe-finance.io/) dưới dạng tệp JSON hoặc có thể tạo bằng cách sử dụng [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli#readme) bằng lệnh `marlowe-cli template`.

Đặt số tiền ADA thành 294 và số tiền đô la thành 100.000000.

In [8]:
ADA=1000000  # 1 ada = 1,000,000 lovelace
LOVELACE_AMOUNT=$((294 * ADA))
MICROUSD_AMOUNT=$((100 * 1000000))
echo "LOVELACE_AMOUNT = $LOVELACE_AMOUNT lovelace"
echo "MICROUSD_AMOUNT = $MICROUSD_AMOUNT µUSD"

LOVELACE_AMOUNT = 294000000 lovelace
MICROUSD_AMOUNT = 100000000 µUSD


Trên blockchain Cardano, các tham số giao thức yêu cầu mỗi UTxO chứa ít nhất một số ADA. Ở đây chúng ta sẽ bắt đầu hợp đồng với 2 ADA.

In [9]:
MIN_LOVELACE="$((2 * ADA))"
echo "MIN_LOVELACE = $MIN_LOVELACE lovelace"

MIN_LOVELACE = 2000000 lovelace


Ở phần sau của ví dụ, chúng ta sẽ cần một số hằng số để chuyển đổi thời gian.

In [10]:
SECOND=1000 # 1 second = 1000 milliseconds
MINUTE=$((60 * SECOND)) # 1 minute = 60 seconds
HOUR=$((60 * MINUTE)) # 1 hour = 60 minutes

Bây giờ hãy thiết kế hợp đồng.

1. Truy cập https://play.marlowe-finance.io/ trong trình duyệt web.
2. Chọn "Open an Example".
3. Chọn "Marlowe" hoặc "Blockly" trong "Swap".
4. Chỉnh sửa hợp đồng để đô la token có ID chính sách và tên cho đô la token. Trong ví dụ này, chúng ta sử dụng test djed, có ID chính sách `9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe` và tên token `Djed_testMicroUSD`.
5. Chọn "Send to Simulator".
6. Đặt "Timeout for ada deposit" thành một giờ trong tương lai.
7. Đặt "Timeout for dollar deposit" thành hai giờ trong tương lai
8. Đặt "Amount of Ada" thành 294.
9. Đặt "Amount of dollars" thành 100.000.000, vì đơn vị đo lường là phần triệu.
10. Chọn "Download as JSON", đặt tên tệp thành "swap-contract.json", và lưu trữ tệp trong thư mục này, cụ thể là [marlowe-starter-kit/](.).

![Setting parameters for the swap contract in Marlowe Playground](images/swap-simulation.png)

## Kiểm tra hợp đồng

Xem tệp hợp đồng dưới dạng YAML.

In [11]:
json2yaml swap-contract.json

timeout: 1679669100000
timeout_continuation: close
when:
- case:
    deposits:
      multiply: 1000000
      times: 294
    into_account:
      role_token: Ada provider
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      role_token: Ada provider
  then:
    timeout: 1679673000000
    timeout_continuation: close
    when:
    - case:
        deposits: 100000000
        into_account:
          role_token: Dollar provider
        of_token:
          currency_symbol: 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe
          token_name: Djed_testMicroUSD
        party:
          role_token: Dollar provider
      then:
        from_account:
          role_token: Ada provider
        pay:
          multiply: 1000000
          times: 294
        then:
          from_account:
            role_token: Dollar provider
          pay: 100000000
          then: close
          to:
            party:
              role_token: Ada provider
          token:
           

### \[Không bắt buộc, nhưng nên dùng\] Kiểm tra mức độ an toàn của hợp đồng

Nếu chúng ta đang chạy hợp đồng trên Cardano `mainnet`, thì chúng ta muốn kiểm tra tính an toàn của nó trước khi tạo nó, để không có khả năng chúng ta có thể mất tiền.

Dưới đây là các bước để kiểm tra mức độ an toàn của hợp đồng:

1. Hiểu [Ngôn ngữ Marlowe](https://marlowe-finance.io/).
2. Hiểu [Mô hình UTxO mở rộng] của Cardano (https://docs.cardano.org/learn/eutxo-explainer).
3. Đọc và hiểu [Hướng dẫn về các phương pháp hay nhất của Marlowe](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Đọc và hiểu [Hướng dẫn bảo mật của Marlowe](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Sử dụng [Marlowe Playground](https://play.marlowe-finance.io/) để gắn cờ cảnh báo, thực hiện phân tích tĩnh và mô phỏng hợp đồng.
6. Sử dụng [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) công cụ `marlowe-cli run analyze` để nghiên cứu xem hợp đồng có thể chạy trên mạng Cardano hay không.
7. Chạy *tất cả đường dẫn thực thi* của hợp đồng trên [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

Ở đây, chúng ta sẽ thực hiện bước 6. Đầu tiên, chúng ta gộp hợp đồng và trạng thái ban đầu của nó vào một tệp duy nhất.

## Giao dịch 1: Nhà cung cấp Ada tạo Hợp đồng hoán đổi với ADA ban đầu

Yêu cầu `HTTP` `POST` đối với điểm cuối `/contracts` của Marlowe Runtime\ sẽ xây dựng giao dịch tạo cho hợp đồng Marlowe. Chúng ta cung cấp cho nó tệp JSON chứa hợp đồng và cho nó biết giá trị `MIN_LOVELACE` mà chúng ta đã chọn trước đó. Bất kỳ ai cũng có thể tạo hợp đồng, nhưng trong ví dụ này, người cho vay sẽ làm như vậy, vì vậy chúng ta cung cấp địa chỉ của họ để tài trợ cho giao dịch và nhận tiền thay đổi từ đó.

Trước tiên, chúng ta tạo phần thân JSON của yêu cầu để xây dựng giao dịch tạo.

In [12]:
yaml2json << EOI > request-1.json
version: v1
contract: `cat swap-contract.json`
roles:
  Ada provider: "$ADA_PROVIDER_ADDR"
  Dollar provider: "$USD_PROVIDER_ADDR"
minUTxODeposit: $MIN_LOVELACE
metadata: {}
tags: {}
EOI
cat request-1.json

{"contract":{"timeout":1679669100000,"timeout_continuation":"close","when":[{"case":{"deposits":{"multiply":1000000,"times":294},"into_account":{"role_token":"Ada provider"},"of_token":{"currency_symbol":"","token_name":""},"party":{"role_token":"Ada provider"}},"then":{"timeout":1679673000000,"timeout_continuation":"close","when":[{"case":{"deposits":100000000,"into_account":{"role_token":"Dollar provider"},"of_token":{"currency_symbol":"9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe","token_name":"Djed_testMicroUSD"},"party":{"role_token":"Dollar provider"}},"then":{"from_account":{"role_token":"Ada provider"},"pay":{"multiply":1000000,"times":294},"then":{"from_account":{"role_token":"Dollar provider"},"pay":100000000,"then":"close","to":{"party":{"role_token":"Ada provider"}},"token":{"currency_symbol":"9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe","token_name":"Djed_testMicroUSD"}},"to":{"party":{"role_token":"Dollar provider"}},"token":{"currency_symbol":"",

Tiếp theo, chúng ta đăng yêu cầu và xem phản hồi.

In [13]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $ADA_PROVIDER_ADDR" \
  -d @request-1.json \
  -o response-1.json \
  -sS
json2yaml response-1.json

links:
  contract: contracts/c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef%231
resource:
  contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  txBody:
    cborHex: 86a8008182582081fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb51010d8182582081fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb51010184a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a3b54cd37a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a001e84800282005820bf610358ef3035e032f179bffee0de88c48478d4aef7b7b2bcb0d3a7351092bda200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc5901821a00102da4a1581cbb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4a14c4164612070726f766964657201a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a101821a00106026a1581cbb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4a14f446f6c6c61722070726f76696465720110a200581d601b120d7221aa5e7db58fd8e0753e156cf8c3

Mã định danh cho hợp đồng được nhúng trong phản hồi.

In [14]:
CONTRACT_ID="$(jq -r '.resource.contractId' response-1.json)"
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1


Việc xê-ri hóa CBOR (ở định dạng phong bì văn bản) cũng được nhúng trong phản hồi.

In [15]:
jq '.resource.txBody' response-1.json > tx-1.unsigned

Có nhiều cách để ký và gửi giao dịch Cardano:
- `cardano-cli` tại dòng lệnh
- `cardano-wallet` tại dòng lệnh hoặc dưới dạng dịch vụ REST
- `cardano-hw-cli` đối với ví phần cứng tại dòng lệnh
- ví CIP-30 tương thích với Babbage trong trình duyệt web
- `marlowe-cli` tại dòng lệnh

Để thuận tiện, ở đây chúng ta sử dụng `marlowe-cli` giao dịch gửi. Người ta có thể phải đợi một phút hoặc lâu hơn để các giao dịch được xác nhận trên blockchain.

In [16]:
TX_1=$(
marlowe-cli transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer "$ADA_PROVIDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"

TX_1 = c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef


Người ta có thể xem giao dịch trên Cardano explorer và thấy rằng hợp đồng đã được tạo và các bên đã nhận được vai trò token của họ. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [17]:
echo "$EXPLORER_URL"/transaction/"$TX_1?tab=utxo"

https://preprod.cardanoscan.io/transaction/c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef?tab=utxo


Cụ thể, chúng ta thấy rằng hợp đồng Marlowe giữ 2 ADA đã được đặt là `MINIMUM_LOVELACE`.

In [18]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$CONTRACT_ID"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "bf610358ef3035e032f179bffee0de88c48478d4aef7b7b2bcb0d3a7351092bd"


Có thể thấy rằng các nhà cung cấp ADA và đô la đã nhận được vai trò token của họ. Lưu ý rằng `4164612070726f7669646572 = "Ada provider`, `4275796572 = Buyer`, và `446f6c6c61722070726f7669646572 = "Dollar provider"` ở dạng ký hiệu thập lục phân. Cũng lưu ý rằng `446a65645f746573744d696372 = Djed_testMicroUSD`.

In [19]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADA_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef     0        995413303 lovelace + TxOutDatumNone
c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef     2        1060260 lovelace + 1 bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4.4164612070726f7669646572 + TxOutDatumNone


In [20]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$USD_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
81fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb51     2        1000000000 lovelace + TxOutDatumNone
81fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb51     3        2000000 lovelace + 100000000 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe.446a65645f746573744d6963726f555344 + TxOutDatumNone
c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef     3        1073190 lovelace + 1 bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4.446f6c6c61722070726f7669646572 + TxOutDatumNone


## Xem chi tiết của hợp đồng trên blockchain

Marlowe Runtime\'s `HTTP` `GET` điểm cuối `/contracts/{contractId}` có thể tìm nạp hợp đồng từ blockchain và trả lại thông tin về hợp đồng đó.

In [21]:
CONTRACT_URL="$MARLOWE_RT_WEBSERVER_URL/`jq -r '.links.contract' response-1.json`"
echo "CONTRACT_URL = $CONTRACT_URL"

CONTRACT_URL = http://127.0.0.1:3780/contracts/c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef%231


In [22]:
curl -sS "$CONTRACT_URL" | json2yaml

links:
  transactions: contracts/c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef%231/transactions
resource:
  block:
    blockHeaderHash: 65a52894d0f20909681734f5b5f461e0e24fdc79c9115b8f2b127856ed754378
    blockNo: 757491
    slotNo: 23982361
  continuations: null
  contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  currentContract:
    timeout: 1679669100000
    timeout_continuation: close
    when:
    - case:
        deposits:
          multiply: 1000000
          times: 294
        into_account:
          role_token: Ada provider
        of_token:
          currency_symbol: ''
          token_name: ''
        party:
          role_token: Ada provider
      then:
        timeout: 1679673000000
        timeout_continuation: close
        when:
        - case:
            deposits: 100000000
            into_account:
              role_token: Dollar provider
            of_token:
              currency_symbol: 9772ff715b691c0444f333ba1db

## Giao dịch 2: Nhà cung cấp Ada gửi 294 Ada vào Hợp đồng

Nhà cung cấp ADA gửi 294 ADA của họ vào hợp đồng bằng cách sử dụng `HTTP` `POST` `/contract/{contractId}/transactions` của Marlowe Runtime\'s . Người mua đang cung cấp tiền cho và nhận tiền thay đổi từ giao dịch này, vì vậy chúng ta cung cấp địa chỉ của họ.

Khoản tiền gửi được thể hiện dưới dạng đầu vào JSON cho hợp đồng. Công cụ `marlowe-cli input Deposit` định dạng JSON chính xác cho một khoản tiền gửi một cách tiện lợi.

In [23]:
marlowe-cli input deposit \
  --deposit-party 'Ada provider' \
  --deposit-account 'Ada provider' \
  --deposit-amount "$LOVELACE_AMOUNT" \
  --out-file input-2.json
json2yaml input-2.json

input_from_party:
  role_token: Ada provider
into_account:
  role_token: Ada provider
of_token:
  currency_symbol: ''
  token_name: ''
that_deposits: 294000000


In [24]:
yaml2json << EOI > request-2.json
version: v1
inputs: [$(cat input-2.json)]
metadata: {}
tags: {}
EOI
cat request-2.json

{"inputs":[{"input_from_party":{"role_token":"Ada provider"},"into_account":{"role_token":"Ada provider"},"of_token":{"currency_symbol":"","token_name":""},"that_deposits":294000000}],"metadata":{},"tags":{},"version":"v1"}


Tiếp theo, chúng ta đăng yêu cầu và lưu trữ phản hồi.

In [25]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $ADA_PROVIDER_ADDR" \
  -d @request-2.json \
  -o response-2.json \
  -sS
json2yaml response-2.json

links:
  transaction: contracts/c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef%231/transactions/84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58
resource:
  contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  transactionId: 84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58
  txBody:
    cborHex: 86aa0083825820c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef00825820c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef01825820c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef020d81825820c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef0012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010183a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a29c39bd0a300581d702ed2631dbb277c84334453c5c437b86325d371f0835a28b910a91a6e011a11a49a000282005820c945bedf448b262fe3e5c36cf3033a4a4914456277f0de38432c4f28bd352ed3a200581d601b120

Một lần nữa, hãy sử dụng `marlowe-cli` để gửi giao dịch rồi chờ xác nhận.

In [26]:
jq '.resource.txBody' response-2.json > tx-2.unsigned

In [27]:
TX_2=$(
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$ADA_PROVIDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_2 = $TX_2"

TX_2 = 84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58


Người ta có thể xem giao dịch trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [28]:
echo "$EXPLORER_URL"/transaction/"$TX_2?tab=utxo"

https://preprod.cardanoscan.io/transaction/84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58?tab=utxo


Có thể thấy rằng nhà cung cấp ADA có ít hơn khoảng 294 ADA so với ban đầu.

In [29]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADA_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58     0        700685264 lovelace + TxOutDatumNone
84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58     2        1060260 lovelace + 1 bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4.4164612070726f7669646572 + TxOutDatumNone


Hợp đồng Marlowe vẫn có 2 ADA từ khi tạo ra nó và thêm 294 ADA.

In [30]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58     1        296000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "c945bedf448b262fe3e5c36cf3033a4a4914456277f0de38432c4f28bd352ed3"


## Xem thêm tiến trình của hợp đồng trên blockchain

Marlowe Runtime\'s `HTTP` `GET` điểm cuối `/contracts/{contractId}/transactions/{transactionId}` có thể tìm nạp hợp đồng từ blockchain và trả lại thông tin về hợp đồng đó.

In [31]:
curl -sS "$CONTRACT_URL"/transactions/"$TX_2" | json2yaml

links: {}
resource:
  block:
    blockHeaderHash: a8e43ba8381f9c177cb788476bb3143ae122b9251e04d5f6fa74260a9b36e122
    blockNo: 757501
    slotNo: 23982522
  consumingTx: null
  continuations: null
  contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  inputUtxo: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  inputs:
  - input_from_party:
      role_token: Ada provider
    into_account:
      role_token: Ada provider
    of_token:
      currency_symbol: ''
      token_name: ''
    that_deposits: 294000000
  invalidBefore: 2023-03-24T13:48:07Z
  invalidHereafter: 2023-03-24T14:45:00Z
  metadata: {}
  outputContract:
    timeout: 1679673000000
    timeout_continuation: close
    when:
    - case:
        deposits: 100000000
        into_account:
          role_token: Dollar provider
        of_token:
          currency_symbol: 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe
          token_name: Djed_testMicroUSD
        party:
 

## Giao dịch 3: Nhà cung cấp Dollar gửi 100 Djed vào Hợp đồng

Nhà cung cấp đô la gửi 100 djed của họ vào hợp đồng bằng cách sử dụng điểm cuối `HTTP` `POST` `/contract/{contractId}/transactions` của Marlowe Runtime\'s. Người mua đang cung cấp tiền cho và nhận tiền thay đổi từ giao dịch này, vì vậy chúng ta cung cấp địa chỉ của họ.

Khoản tiền gửi được thể hiện dưới dạng đầu vào JSON cho hợp đồng. Công cụ `marlowe-cli input Deposit` định dạng JSON chính xác cho một khoản tiền gửi một cách tiện lợi.

In [32]:
marlowe-cli input deposit \
  --deposit-party 'Dollar provider' \
  --deposit-account 'Dollar provider' \
  --deposit-token 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe.Djed_testMicroUSD \
  --deposit-amount "$MICROUSD_AMOUNT" \
  --out-file input-3.json
json2yaml input-3.json

input_from_party:
  role_token: Dollar provider
into_account:
  role_token: Dollar provider
of_token:
  currency_symbol: 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe
  token_name: Djed_testMicroUSD
that_deposits: 100000000


In [33]:
yaml2json << EOI > request-3.json
version: v1
inputs: [$(cat input-3.json)]
metadata: {}
tags: {}
EOI
cat request-3.json

{"inputs":[{"input_from_party":{"role_token":"Dollar provider"},"into_account":{"role_token":"Dollar provider"},"of_token":{"currency_symbol":"9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe","token_name":"Djed_testMicroUSD"},"that_deposits":100000000}],"metadata":{},"tags":{},"version":"v1"}


Tiếp theo, chúng ta đăng yêu cầu và lưu trữ phản hồi.

In [34]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $USD_PROVIDER_ADDR" \
  -d @request-3.json \
  -o response-3.json \
  -sS
json2yaml response-3.json

links:
  transaction: contracts/c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef%231/transactions/05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e
resource:
  contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  transactionId: 05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e
  txBody:
    cborHex: 86aa008482582081fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb510282582081fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb510382582084e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e5801825820c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef030d8182582081fdeb10a575ba9998c1229b1b2b6a4006acf4a641a66e13bd787b769cd8fb510212818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd010185a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a1011a3b98370aa200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a101821a00106026a1581cbb7053ce0

Một lần nữa, hãy sử dụng `marlowe-cli` để gửi giao dịch rồi đợi xác nhận.

In [35]:
jq '.resource.txBody' response-3.json > tx-3.unsigned

In [36]:
TX_3=$(
marlowe-cli transaction submit \
  --tx-body-file tx-3.unsigned \
  --required-signer "$USD_PROVIDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_3 = $TX_3"

TX_3 = 05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e


Người ta có thể xem giao dịch trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [37]:
echo "$EXPLORER_URL"/transaction/"$TX_3?tab=utxo"

https://preprod.cardanoscan.io/transaction/05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e?tab=utxo


Người ta có thể thấy rằng nhà cung cấp đô la đã giảm chính xác 100 djed so với ban đầu.

In [38]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$USD_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e     0        999831306 lovelace + TxOutDatumNone
05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e     1        1073190 lovelace + 1 bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4.446f6c6c61722070726f7669646572 + TxOutDatumNone


Hợp đồng Marlowe đã kết thúc, nhưng địa chỉ thanh toán cuộn vẫn giữ 100 djed vì lợi ích của nhà cung cấp ADA.

In [39]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_3#2"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e     2        1258520 lovelace + 100000000 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe.446a65645f746573744d6963726f555344 + TxOutDatumHash ScriptDataInBabbageEra "758d98b7ec8818a2cd32baa416d6a337bcb837a2393815744e3f4935bebedd3e"


Địa chỉ thanh toán cuộn cũng giữ 294 ADA vì lợi ích của nhà cung cấp đô la.

In [40]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_3#3"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e     3        294000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "366c6e0ba1da1c0fe3488c49fba1f8dcdd6f4dcc723bbd08bc62caf12633facc"


## Xem thêm tiến trình của hợp đồng trên blockchain

Marlowe Runtime\'s HTTP GET điểm cuối /contracts/{contractId}/transactions/{transactionId} có thể tìm nạp hợp đồng từ blockchain và trả lại thông tin về hợp đồng đó.

In [41]:
curl -sS "$CONTRACT_URL"/transactions/"$TX_3" | json2yaml

links:
  previous: contracts/c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef%231/transactions/84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58
resource:
  block:
    blockHeaderHash: c331402d98a3e77d8d78123a915118e39e4acef61613e57cae46cc047d796fb7
    blockNo: 757509
    slotNo: 23982639
  consumingTx: null
  continuations: null
  contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  inputUtxo: 84e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58#1
  inputs:
  - input_from_party:
      role_token: Dollar provider
    into_account:
      role_token: Dollar provider
    of_token:
      currency_symbol: 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe
      token_name: Djed_testMicroUSD
    that_deposits: 100000000
  invalidBefore: 2023-03-24T13:50:04Z
  invalidHereafter: 2023-03-24T15:50:00Z
  metadata: {}
  outputContract: null
  outputState: null
  outputUtxo: null
  status: confirmed
  tags: {}
  transacti

## Giao dịch 4: Nhà cung cấp Ada rút đô la

100 djed được giữ tại địa chỉ thanh toán theo vai trò của Marlowe vì lợi ích của nhà cung cấp ADA. Nhà cung cấp ADA có thể rút các khoản tiền này bất cứ lúc nào. ID hợp đồng và tên vai trò được bao gồm trong nội dung yêu cầu rút tiền.

In [42]:
yaml2json << EOI > request-4.json
contractId: "$CONTRACT_ID"
role: "Ada provider"
EOI
cat request-4.json

{"contractId":"c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1","role":"Ada provider"}


Tiếp theo, chúng ta đăng yêu cầu và lưu trữ phản hồi.

In [43]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $ADA_PROVIDER_ADDR" \
  -d @request-4.json \
  -o response-4.json \
  -sS
json2yaml response-4.json

links:
  withdrawal: withdrawals/cdd5691e3085d6dd50ac45911b2a6bbd372a232a05a68851263279a568973a37
resource:
  txBody:
    cborHex: 86a8008382582005b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e0282582084e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e580082582084e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e58020d8182582084e284b1ad405c2b32e4eb4d93c31ec15b10470737716f3461ed0623e8572e580012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd020183a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc59011a29c08003a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc5901821a00102da4a1581cbb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4a14c4164612070726f766964657201a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38742546a907e8104cc5901821a0010c52aa1581c9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fea151446a65645f746573744d6963726f5553441a05f5e10010a200581d601b120d7221aa5e7db58fd8e0753e156cf8c38

Một lần nữa, hãy sử dụng `marlowe-cli` để gửi giao dịch rồi đợi xác nhận.

In [44]:
jq '.resource.txBody' response-4.json > tx-4.unsigned

In [45]:
TX_4=$(
marlowe-cli transaction submit \
  --tx-body-file tx-4.unsigned \
  --required-signer "$ADA_PROVIDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_4 = $TX_4"

TX_4 = cdd5691e3085d6dd50ac45911b2a6bbd372a232a05a68851263279a568973a37


Bạn có thể xem giao dịch trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [46]:
echo "$EXPLORER_URL"/transaction/"$TX_4?tab=utxo"

https://preprod.cardanoscan.io/transaction/cdd5691e3085d6dd50ac45911b2a6bbd372a232a05a68851263279a568973a37?tab=utxo


Nhà cung cấp ADA hiện có 100 djed.

In [47]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$ADA_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e     4        2000000 lovelace + TxOutDatumNone
cdd5691e3085d6dd50ac45911b2a6bbd372a232a05a68851263279a568973a37     0        700481539 lovelace + TxOutDatumNone
cdd5691e3085d6dd50ac45911b2a6bbd372a232a05a68851263279a568973a37     1        1060260 lovelace + 1 bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4.4164612070726f7669646572 + TxOutDatumNone
cdd5691e3085d6dd50ac45911b2a6bbd372a232a05a68851263279a568973a37     2        1099050 lovelace + 100000000 9772ff715b691c0444f333ba1db93b055c0864bec48fff92d1f2a7fe.446a65645f746573744d6963726f555344 + TxOutDatumNone


## Xem giao dịch rút tiền

Marlowe Runtime\'s `HTTP` `GET` điểm cuối `/drawals/{transactionId}` có thể tìm nạp một giao dịch rút tiền từ blockchain và trả lại thông tin về giao dịch đó.

In [48]:
curl -sS "$MARLOWE_RT_WEBSERVER_URL"/withdrawals/"$TX_4" | json2yaml

block:
  blockHeaderHash: b41c4d2ed3de99126ddfbd379c6e6f4387cbf5bbc8af744a04b66754280bd3ed
  blockNo: 757514
  slotNo: 23982857
payouts:
- contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  payout: 05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e#2
  role: Ada provider
  roleTokenMintingPolicyId: bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4
status: confirmed
withdrawalId: cdd5691e3085d6dd50ac45911b2a6bbd372a232a05a68851263279a568973a37


## Giao dịch 5: Nhà cung cấp Dollar rút Ada

294 ADA được giữ tại địa chỉ thanh toán theo vai trò của Marlowe vì lợi ích của nhà cung cấp đô la. Nhà cung cấp đô la có thể rút các khoản tiền này bất cứ lúc nào. ID hợp đồng và tên vai trò được bao gồm trong nội dung yêu cầu rút tiền.

In [49]:
yaml2json << EOI > request-5.json
contractId: "$CONTRACT_ID"
role: "Dollar provider"
EOI
cat request-5.json

{"contractId":"c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1","role":"Dollar provider"}


Tiếp theo, chúng ta đăng yêu cầu và lưu trữ phản hồi.

In [50]:
curl "$MARLOWE_RT_WEBSERVER_URL/withdrawals" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $USD_PROVIDER_ADDR" \
  -d @request-5.json \
  -o response-5.json \
  -sS
json2yaml response-5.json

links:
  withdrawal: withdrawals/315f54a7c39c96f1bcc521954edc5e33b378d3aebfc71f2a1168d358adef5867
resource:
  txBody:
    cborHex: 86a8008382582005b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e0082582005b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e0182582005b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e030d8182582005b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e0012818258209a8a6f387a3330b4141e1cb019380b9ac5c72151c0abc52aa4266245d3c555cd020182a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a1011a4d193a50a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a101821a00106026a1581cbb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4a14f446f6c6c61722070726f76696465720110a200581d604959d439bd61c63c94a102be0defd6a54b36071d6789f05b7b0224a1011a3b909bb3111a00079b57021a0005123a0b5820424a94276d7c3699508f073d8809325917434fbf05bd0b48f82aba66a55997c49fff81d8799f581cbb7053ce07dda2292f779c92ae789b4c4a99376f5f6

Một lần nữa, hãy sử dụng `marlowe-cli` để gửi giao dịch rồi chờ xác nhận.

In [51]:
jq '.resource.txBody' response-5.json > tx-5.unsigned

In [52]:
TX_5=$(
marlowe-cli transaction submit \
  --tx-body-file tx-5.unsigned \
  --required-signer "$USD_PROVIDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_5 = $TX_5"

TX_5 = 315f54a7c39c96f1bcc521954edc5e33b378d3aebfc71f2a1168d358adef5867


Bạn có thể xem giao dịch trên Cardano explorer. Đôi khi phải mất ba mươi giây hoặc lâu hơn để giao dịch hiển thị trong explorer.

In [53]:
echo "$EXPLORER_URL"/transaction/"$TX_5?tab=utxo"

https://preprod.cardanoscan.io/transaction/315f54a7c39c96f1bcc521954edc5e33b378d3aebfc71f2a1168d358adef5867?tab=utxo


Nhà cung cấp đô la hiện có thêm khoảng 294 ADA.

In [54]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$USD_PROVIDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
315f54a7c39c96f1bcc521954edc5e33b378d3aebfc71f2a1168d358adef5867     0        1293498960 lovelace + TxOutDatumNone
315f54a7c39c96f1bcc521954edc5e33b378d3aebfc71f2a1168d358adef5867     1        1073190 lovelace + 1 bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4.446f6c6c61722070726f7669646572 + TxOutDatumNone


## Xem rút tiền

Marlowe Runtime\'s `HTTP` `GET` điểm cuối `/drawals/{transactionId}` có thể tìm nạp một khoản rút tiền từ chuỗi khối và trả về thông tin về nó.

In [55]:
curl -sS "$MARLOWE_RT_WEBSERVER_URL"/withdrawals/"$TX_5" | json2yaml

block:
  blockHeaderHash: f4a1cfc8471034fcc5f58008909c429e8f9d0bdd8efdb93ab29d2e62b7a0bfa0
  blockNo: 757519
  slotNo: 23982974
payouts:
- contractId: c73c08a937cc0244abc28224ee0bd8fddef4c3315a74a9207776dfed87ff11ef#1
  payout: 05b05966afb5d346d3f2a03470df632b5b697265f637d2adb5fcb463bd127f5e#3
  role: Dollar provider
  roleTokenMintingPolicyId: bb7053ce07dda2292f779c92ae789b4c4a99376f5f6d92b5c4a9f6a4
status: confirmed
withdrawalId: 315f54a7c39c96f1bcc521954edc5e33b378d3aebfc71f2a1168d358adef5867
